In [15]:
import tensorflow as tf

In [ ]:
import tensorflow_datasets as tfds

In [ ]:
imdb, info = tfds.load('imdb_reviews',with_info=True,as_supervised=True)

tfds.core.DatasetInfo(
    name='imdb_reviews',
    version=0.1.0,
    description='Large Movie Review Dataset.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.',
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=100000,
    splits={
        'test': 25000,
        'train': 25000,
        'unsupervised': 50000,
    },
    supervised_keys=('text', 'label'),
    citation="""@InProceedings{maas-EtAl:2011:ACL-HLT2011,
      author    = {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Potts, Christopher},
      title     = {Learning Word

In [6]:
train_data, test_data = imdb['train'], imdb['test']

In [27]:
training_sentences = []
testing_sentences = []

training_labels =[]
testing_labels =[]

for s,l in train_data:
    training_sentences.append(str(s.numpy()))
    training_labels.append(l.numpy())
for s,l in test_data:
    testing_sentences.append(str(s.numpy()))
    testing_labels.append(l.numpy())

In [28]:
import numpy as np
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [29]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok = '<OOV>'

In [30]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

train_seq = tokenizer.texts_to_sequences(training_sentences)
train_seq = pad_sequences(train_seq, maxlen=max_length,truncating=trunc_type)

test_seq = tokenizer.texts_to_sequences(testing_sentences)
test_seq = pad_sequences(test_seq, maxlen=max_length)


In [31]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(train_seq[1]))
print(training_sentences[1])

b oh yeah jenna jameson did it again yeah baby this movie rocks it was one of the 1st movies i saw of her and i have to say i feel in love with her she was great in this move br br her performance was outstanding and what i liked the most was the scenery and the wardrobe it was amazing you can tell that they put a lot into the movie the girls cloth were amazing br br i hope this comment helps and u can buy the movie the storyline is awesome is very unique and i'm sure u are going to like it jenna amazed us once more and no wonder the movie won so many
b"Oh yeah! Jenna Jameson did it again! Yeah Baby! This movie rocks. It was one of the 1st movies i saw of her. And i have to say i feel in love with her, she was great in this move.<br /><br />Her performance was outstanding and what i liked the most was the scenery and the wardrobe it was amazing you can tell that they put a lot into the movie the girls cloth were amazing.<br /><br />I hope this comment helps and u can buy the movie, the

In [32]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length = max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss = 'binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [34]:
num_epochs = 10
model.fit(train_seq, training_labels_final, epochs=num_epochs, validation_data=(test_seq, testing_labels_final))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 2s 89us/sample - loss: 0.4946 - accuracy: 0.7434 - val_loss: 0.3672 - val_accuracy: 0.8371
Epoch 2/10
25000/25000 [==============================] - 2s 80us/sample - loss: 0.2346 - accuracy: 0.9094 - val_loss: 0.3829 - val_accuracy: 0.8325
Epoch 3/10
25000/25000 [==============================] - 2s 80us/sample - loss: 0.0862 - accuracy: 0.9788 - val_loss: 0.4734 - val_accuracy: 0.8204
Epoch 4/10
25000/25000 [==============================] - 2s 79us/sample - loss: 0.0219 - accuracy: 0.9973 - val_loss: 0.5521 - val_accuracy: 0.8185
Epoch 5/10
25000/25000 [==============================] - 2s 82us/sample - loss: 0.0049 - accuracy: 0.9998 - val_loss: 0.6076 - val_accuracy: 0.8238
Epoch 6/10
25000/25000 [==============================] - 2s 78us/sample - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.6545 - val_accuracy: 0.8233
Epoch 7/10
25000/25000 [==============================] 

In [38]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [41]:
#Can be used to visualise the words in Tensorboard.
#Open projector.tensorflow.org -> Load.
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()